In this assignment, you will be addressing one application of recurrent neural networks (RNN), implementing in Python a multi-layer RNN for training/sampling from character-level language models. The model takes one text file as input and trains an RNN to learn to predict the next character in a sequence. The RNN can then be used to generate text character by character that will look like the original training data. The echo state network is addressed, as well as long short-term memory and gate recurrent units.

Get the available code “min-char-rnn.py” and “grad-check.py”. Using any IDE environment of your choosing, copy and paste the code. Download the character-level RNN, "karpathy/char-rnn," located in the Topic 7 Study Materials, and train it on the “tiny Shakespeare” data set available at the same location. Consider the following:

1. What is the RNN architecture used for reading the code “min-char-rnn.py”?
2. Create outputs of the language model after training for 5 epochs.
3. Create outputs of the language model after training for 50 epochs.
4. Create outputs of the language model after training for 500 epochs.
5. Check the gradient descent using the code “grad-check.py”.
6. What significant differences do you see between the three outputs?
7. What are the challenges encountered for training the RNN?

The Python code necessary for this assignment is available within the zip file "DSC-550-Python-Project-Files," located in the Course Materials.

APA style is not required, but solid academic writing is expected. 

This assignment uses a rubric. Please review the rubric prior to beginning the assignment to become familiar with the expectations for successful completion. 

Submit a Microsoft Word document with the source code, as directed by your instructor.

Zip your document and submit to LoudCloud or GitHub, as directed by your instructor. Be sure to include the GitHub link in your Word document if submitting via GitHub.

You are required to submit this assignment to LopesWrite. Refer to the LopesWrite Technical Support articles for assistance. 

In [23]:
# gradient checking
from random import uniform

def gradCheck(inputs, target, hprev):

    global Wxh, Whh, Why, bh, by
    num_checks, delta = 10, 1e-5
    _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFun(inputs, targets, hprev)
    for param,dparam,name in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], ['Wxh', 'Whh', 'Why', 'bh', 'by']):
        s0 = dparam.shape
        s1 = param.shape
        assert s0 == s1
#     assert s0 == s1, 'Error dims dont match: %s and %s.' % (`s0`, `s1`)
        print(name)
    for i in range(num_checks):
        ri = int(uniform(0,param.size))
        # evaluate cost at [x + delta] and [x - delta]
        old_val = param.flat[ri]
        param.flat[ri] = old_val + delta
        cg0, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
        param.flat[ri] = old_val - delta
        cg1, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
        param.flat[ri] = old_val # reset old value for this parameter
        # fetch both numerical and analytic gradient
        grad_analytic = dparam.flat[ri]
        grad_numerical = (cg0 - cg1) / ( 2 * delta )
        rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
        print('%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
#         print(rel_error)
        # rel_error should be on order of 1e-7 or less

In [24]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
        
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

print('GRADIENT CHECK: ')
print(gradCheck(inputs, targets, hprev))
    
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n == 5 or n == 50 or n == 500 or n == 5000 or n == 50000:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n == 5 or n == 50 or n == 500 or n == 5000 or n == 50000: 
        print('iter %d, loss: %f' % (n, smooth_loss))  # print progress
#         print('iter %d, Grad: %f' % (n, gradCheck(inputs, targets, hprev)))  # print grad
        
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 
    if n == 50001:
        break
    
#     if n == 5 or n == 50 or n == 500 or n == 5000 or n == 50000 or n == 500000: 
#         print('iter %d, grad output: %f' % (n, gradCheck(inputs, targets, hprev)))  # print grad


data has 1115394 characters, 65 unique.
GRADIENT CHECK: 
Wxh
Whh
Why
bh
by
0.384685, 0.384685 => 2.888849e-09 
0.384810, 0.384810 => 1.732522e-09 
0.384495, 0.384495 => 1.075574e-09 
0.384583, 0.384583 => 1.141845e-09 
0.384420, 0.384420 => 2.233834e-09 
0.384710, 0.384710 => 6.889657e-10 
0.384598, 0.384598 => 1.290958e-09 
0.384648, 0.384648 => 8.646096e-10 
0.384666, 0.384666 => 2.762385e-10 
0.384896, 0.384896 => 3.300201e-09 
None
----
 vh wdhdhrh l ajl l h a l l hvl lololdGol a hdlvl ldl h l c l l a adhoh ldhohoEdldavaol loldOoholkh h VoL h hdholoh l hva h hoh holoPdl hoadl l l l l Yehvlda lvlvldLdldldldlolvlolo: ldiolol loldholvlol 
----
iter 5, loss: 104.748499
----
 nyedefo; d veyev ytdltiyezsheoef
 ka  rn u,keutOe
ndeue ralR arrjweoeueu;;ib?aidiItcncmyebedec  ebluocefiuta afnhdeo d neyes ots
Eed t ;soi!tpA;esg  ;ey .td zebennu M 'ezeu dlue'IfeOso ; ; iesBoi; yew 
----
iter 50, loss: 105.741687
----
 iis mdtrty tal !
wh m  lmiticiui,s  yi thime iauCl
yUOhiSr
u O Omecols z nlnu

When looking at the Gradient descentoutput, it appears that one of the most common challenges with RNNs has occured in this analysis as well - there is vanishing gradients. When training a vanilla RNN using back-propagation, the gradients which are back-propagated can vanish or explode(0 or infinity) because of the computations involved in the process, which use finite-precision numbers. RNNs using LSTM units partially solve the vanishing gradient problem, because LSTM units allow gradients to also flow unchanged. However, LSTM networks can still suffer from the exploding gradient problem. In a next iteration of this analysis we could eplore integrating LSTM or GRU methods.